In [1]:
# Explicação resumida do problema:
# 03 fábricas → cada uma produz 2000 toneladas.
# 03 depósitos → com capacidades 3000, 3000 e 1500 toneladas.
# Custos por tonelada são dados no gráfico.

In [4]:
# Estudante: Luan Bispo Silva
# RGM: 32635648
# Matéria: Computabilidade e Complexidade de Algoritmos.

In [6]:
!pip install pulp

In [8]:
# Problema de transporte - minimização de custos logísticos
# Importa a biblioteca de programação linear
import pulp

# Fábricas e suas produções mensais (toneladas)
fabricas = ['F1', 'F2', 'F3']
oferta = {'F1': 2000, 'F2': 2000, 'F3': 2000}

# Depósitos e suas capacidades mensais (toneladas)
depositos = ['D1', 'D2', 'D3']
demanda = {'D1': 3000, 'D2': 3000, 'D3': 1500}

# Custos de transporte (R$/tonelada)
custos = {
    ('F1', 'D1'): 4, ('F1', 'D2'): 2, ('F1', 'D3'): 6,
    ('F2', 'D1'): 8, ('F2', 'D2'): 2, ('F2', 'D3'): 4,
    ('F3', 'D1'): 4, ('F3', 'D2'): 4, ('F3', 'D3'): 4
}

# Cria o modelo de minimização
modelo = pulp.LpProblem("problema_de_transporte", pulp.LpMinimize)

# Variáveis de decisão
# x[(i,j)] representa a quantidade enviada da fábrica i ao depósito j
x = pulp.LpVariable.dicts("x", custos.keys(), lowBound=0, cat='Continuous')

# Função objetivo
# Minimizar o custo total de transporte
modelo += pulp.lpSum([custos[(i, j)] * x[(i, j)] for (i, j) in custos]), "custo_total"

# Restrições de oferta
# Aqui assumimos que cada fábrica envia toda a sua produção (igualdade)
for i in fabricas:
    modelo += pulp.lpSum([x[(i, j)] for j in depositos]) == oferta[i], f"oferta_{i}"

# Restrições de demanda (capacidade máxima dos depósitos)
# cada depósito recebe até sua capacidade (não é obrigatório receber tudo)
for j in depositos:
    modelo += pulp.lpSum([x[(i, j)] for i in fabricas]) <= demanda[j], f"demanda_{j}"

# Resolução do modelo
resultado = modelo.solve(pulp.PULP_CBC_CMD(msg=0))

# Resultados e diagnóstico
print("Status da solução:", pulp.LpStatus[modelo.status])
if pulp.LpStatus[modelo.status] != 'Optimal':
    print("aviso: solução não ótima. verifique balanço oferta/demanda ou restrições adicionais.")
print("\nRotas e quantidades transportadas:")
for (i, j) in custos:
    val = x[(i, j)].varValue
    if val is not None and val > 0:
        print(f"  {i} → {j}: {val:.0f} t")

print(f"\nCusto total mínimo: R$ {pulp.value(modelo.objective):,.2f}")

Status da solução: Optimal

Rotas e quantidades transportadas:
  F1 → D2: 2000 t
  F2 → D2: 1000 t
  F2 → D3: 1000 t
  F3 → D1: 2000 t

Custo total mínimo: R$ 18,000.00
